When using Google Colab, upload the compressed datasets and run the cell below to decompress the dataset

---



In [ ]:
# import zipfile
# with zipfile.ZipFile("processed-data-labeled.zip","r") as zip_ref:
#     zip_ref.extractall("./")
# with zipfile.ZipFile("processed-data-unlabeled.zip","r") as zip_ref:
#     zip_ref.extractall("./")

Import libraries

In [ ]:
import csv
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Activation, Flatten, Input, Concatenate
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow_addons as tfa 
import matplotlib.pyplot as plt

Global hyperparameters and constants

In [ ]:
threshold = 0.01
hidden_size = 100
word_embedding_dim = 300
epochs = 10
use_unlabeled_dataset = True

labeled_dataset_size = 1830
train_dataset_size = 900
validation_dataset_size = 100
test_dataset_size = 830
unlabeled_dataset_size = 4000

pos_list = np.char.lower(["ADJ","ADP","ADV","AUX","CONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"])
dep_list = np.char.lower(["ROOT", "acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nmod", "npadvmod", "nsubj", "nsubjpass", "nummod", "oprd", "parataxis", "pcomp", "pobj", "poss", "preconj", "predet", "prep", "prt", "punct", "quantmod", "relcl", "xcomp"])

pos_dim = len(pos_list)
dep_dim = len(dep_list)

# Load Data

In [ ]:
# Helper function to one-hot encode the labels
def one_hot(vec, dic):
    vec = np.char.lower(vec)
    return np.array([dic == row for row in vec], dtype='i1')

In [ ]:
labeled_dataset = []
unlabeled_dataset = []

train_dataset = []
validation_dataset = []
test_dataset = []
ul_dataset = []

# Read labeled dataset
for i in range(1, labeled_dataset_size + 1):
    filename = "new_processed-data-labeled/processed-labeled-tweet-{}.csv".format(i)
    if os.path.exists(filename):
        with open(filename, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            data = [tuple(x) for x in spamreader]
            data = np.array(data, dtype=([("text", 'U20'),("simplified_text", 'U20'), ("best_match", 'U20'), ("index", int), ("pos", 'U20'), ("dep", 'U20'), ("stop", 'U5'), ("label", 'i1')]))
            if len(data):
                labeled_dataset.append(data)

# Reshape labeled dataset
for i in range(len(labeled_dataset)):
    tweet = labeled_dataset[i]
    text = tf.reshape(tweet["index"], (1, -1, 1))
    pos = tf.reshape(one_hot(tweet["pos"], pos_list), (1, -1, pos_dim))
    dep = tf.reshape(one_hot(tweet["dep"], dep_list), (1, -1, dep_dim))
    label = tf.reshape(tf.one_hot(tweet["label"], 2), (1, -1, 2))
    train_dataset.append((np.concatenate((text, pos, dep), axis=-1), label))

# Read unlabeled dataset
for i in range(1, unlabeled_dataset_size + 1):
    filename = "processed-data-unlabeled/processed-unlabeled-tweet-{}.csv".format(i)
    if os.path.exists(filename):
        with open(filename, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            data = [tuple(x) for x in spamreader]
            data = np.array(data, dtype=([("text", 'U20'),("simplified_text", 'U20'), ("best_match", 'U20'), ("index", int), ("pos", 'U20'), ("dep", 'U20'), ("stop", 'U5')]))
            if len(data):
                unlabeled_dataset.append(data)

# Reshape unlabeled dataset
for i in range(len(unlabeled_dataset)):
    tweet = unlabeled_dataset[i]
    text = tf.reshape(tweet["index"], (1, -1, 1))
    pos = tf.reshape(one_hot(tweet["pos"], pos_list), (1, -1, pos_dim))
    dep = tf.reshape(one_hot(tweet["dep"], dep_list), (1, -1, dep_dim))
    ul_dataset.append(np.concatenate((text, pos, dep), axis=-1))

# Split labeled dataset
validation_dataset = train_dataset[901:1001]
test_dataset = train_dataset[1001:1101]
train_dataset = train_dataset[:901]

used = np.zeros(len(ul_dataset))

# Define RNN Model

In [ ]:
inputs = Input(shape=(None, pos_dim+dep_dim+1))
x = Embedding(380000, word_embedding_dim)(inputs[:,:,0])
x = Concatenate(axis=-1)([inputs[:,:,1:], x])
x = Bidirectional(LSTM(100, return_sequences=True))(x)
outputs = Dense(2, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

opt = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False
)
model.compile(loss=BinaryCrossentropy(), optimizer=opt)

In [ ]:
# Define precision, recall and F-1 metrics
def metrics(prediction, label):
    prediction = (prediction > 0.5)[0,:,1]
    label = label[0,:,1]
    true_positive = np.sum(np.logical_and((prediction == label), (prediction == True)))
    false_positive = np.sum(np.logical_and((prediction != label), (prediction == True)))
    false_negative = np.sum(np.logical_and((prediction != label), (prediction == False)))
    precision = 0 if true_positive == 0 else true_positive / (true_positive + false_positive)
    recall = 0 if true_positive == 0 else true_positive / (true_positive + false_negative)
    f1 = 0 if true_positive == 0 else 2 / (1 / precision + 1 / recall)
    return precision, recall, f1

# Training

In [ ]:
def train_step(tweet):
    x,y = tweet[0], tweet[1]
    model.fit(x, y, verbose=0)
    prediction = predict_step(x)
    return metrics(prediction, y)

In [ ]:
def eval_step(tweet):
    x,y = tweet[0], tweet[1]
    prediction = predict_step(x)
    return metrics(prediction, y)

In [ ]:
def predict_step(tweet):
    return model.predict(tweet)

In [ ]:
def neg_log(prediction):
    return -np.mean(np.log(np.amax(prediction, axis=-1)))

In [ ]:
train_p, train_r, train_f = [], [], []
val_p, val_r, val_f = [], [], []

def semi_supervised():
    for epoch in range(epochs):

        print("Training epoch {}".format(epoch+1))
        count = 0
        total = 0
        precision, recall, f1 = 0, 0, 0
        for tweet in train_dataset:
            if count % 100 == 0:
                print("Training iter {}".format(count))
            p, r, f = train_step(tweet)
            total += tweet[0].shape[1]
            precision += p * tweet[0].shape[1]
            recall += r * tweet[0].shape[1]
            f1 += f * tweet[0].shape[1]
            count += 1
        train_p.append(precision / total)
        train_r.append(recall / total)
        train_f.append(f1 / total)

        print("Validation")
        total = 0
        precision, recall, f1 = 0, 0, 0
        for tweet in validation_dataset:
            p, r, f = eval_step(tweet)
            total += tweet[0].shape[1]
            precision += p * tweet[0].shape[1]
            recall += r * tweet[0].shape[1]
            f1 += f * tweet[0].shape[1]

        if len(val_f) == 0 or f1 / total > np.amax(val_f):
            model.save_weights('./checkpoint')

        val_p.append(precision / total)
        val_r.append(recall / total)
        val_f.append(f1 / total)

        if use_unlabeled_dataset == True:
            print("Enlarging training set")
            added = 0
            for i in range(len(ul_dataset)):
                tweet = ul_dataset[i]
                if used[i] == False:
                    prediction = predict_step(tweet)
                    if neg_log(prediction) < threshold:
                        label = prediction > 0.5
                        train_dataset.append((tweet, label))
                        used[i] = True
                        added += 1
            
            print("Added {} data points to training set".format(added))

        print(train_p, train_r, train_f)
        print(val_p, val_r, val_f)


    model.load_weights('./checkpoint')
    for tweet in test_dataset:
        p, r, f = eval_step(tweet)
        total += tweet[0].shape[1]
        precision += p * tweet[0].shape[1]
        recall += r * tweet[0].shape[1]
        f1 += f * tweet[0].shape[1]
    print("Test Dataset precision = {}, recall = {}, f1 = {}".format(precision / total, recall / total, f1 / total))


In [ ]:
semi_supervised()

# Plot Results

In [ ]:
# Plot training scores
plt.plot(train_p)
plt.plot(train_r)
plt.plot(train_f)
plt.title('training')
plt.ylabel('score')
plt.xlabel('epoch')
plt.legend(["precision", "recall", "f1"], loc='upper left')
plt.show()

# Plot validation scores
plt.plot(val_p)
plt.plot(val_r)
plt.plot(val_f)
plt.title('validation')
plt.ylabel('score')
plt.xlabel('epoch')
plt.legend(["precision", "recall", "f1"], loc='upper left')
plt.show()
